In [1]:
import gensim
import csv
import logging
import nltk
import numpy as np

In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)  

2017-03-05 13:37:18,056 : INFO : loading projection weights from GoogleNews-vectors-negative300.bin.gz
2017-03-05 13:42:21,379 : INFO : loaded (3000000, 300) matrix from GoogleNews-vectors-negative300.bin.gz


In [4]:
datafile = "data/sarcasm_v2.csv"
with open(datafile) as f:
    data = list(csv.reader(f))[0:]

In [5]:
text = [line[-1].lower() for line in data if line[0]=="GEN"]
text[:3]

['therefore you accept that the republican party almost as a whole is "grade a usda approved liberalism." about time you did.   ',
 "more chattering from the peanut gallery? haven't gotten the memo, you're no longer a player? honestly....clamoring for attention is so low budget. no shame.",
 'yep, suppressing natural behavior is always the way to go. we should also get them to stop pooping.']

In [60]:
test = ["here is a list of strings.", "there are definitely stopwords in it.", "let's see if we can remove them."]

In [6]:
test = [[word for word in test if word not in nltk.corpus.stopwords.words('english')] for line in test]


NameError: name 'test' is not defined

In [15]:
tokens = [nltk.word_tokenize(line) for line in text]
print(tokens[0])

['therefore', 'you', 'accept', 'that', 'the', 'republican', 'party', 'almost', 'as', 'a', 'whole', 'is', '``', 'grade', 'a', 'usda', 'approved', 'liberalism', '.', "''", 'about', 'time', 'you', 'did', '.']


In [15]:
model.doesnt_match('girl boy baby grandma'. split())

'grandma'

In [18]:
model_name = "word2vec"
model.save(model_name)

In [19]:
model.syn0.shape


(3000000, 300)

In [62]:
def makeFeatureVec(doc,model):
    featureVec = np.zeros(300,)
    
    nwords = 0
    
    index2word_set = set(model.index2word)
    
    for word in doc.split():
        print(word)
        if word in index2word_set:
            nwords += 1
            featureVec = np.add(featureVec, model[word])
    
    featureVec = np.divide(featureVec, nwords)
    return featureVec
    

In [67]:
def getAvgFeatureVecs(text, model):
    counter = 0
    
    textFeatureVecs = np.zeros((len(text), 300), dtype="float32")
    for doc in text:
#         if counter%1000 == 0:
#             print(type(counter), type(len(text)))
#             print("doc %d of %d") %(counter, len(doc))
            
        textFeatureVecs[counter] = makeFeatureVec(doc, model)
        counter += 1
    return textFeatureVecs
    

In [68]:
test = getAvgFeatureVecs(text[:1], model)
test

therefore
you
accept
that
the
republican
party
almost
as
a
whole
is
"grade
a
usda
approved
liberalism."
about
time
you
did.


array([[ 0.00562096, -0.032341  ,  0.07038498,  0.12967467, -0.1184082 ,
        -0.01626897,  0.03766251, -0.02767944,  0.06901169,  0.03584671,
        -0.06720734, -0.07981873, -0.06523705,  0.00865173, -0.09159851,
         0.11291599,  0.04623604,  0.04164124,  0.0475893 , -0.06226349,
         0.0039562 ,  0.04541016,  0.01827621,  0.02708197,  0.06001663,
         0.03572083, -0.06299973,  0.06289291,  0.06230259,  0.03058624,
        -0.00534439, -0.02361679, -0.03826523,  0.05932903, -0.01037216,
        -0.0222311 , -0.00365829,  0.02702141, -0.01179504,  0.05965805,
         0.08377075, -0.0976181 ,  0.13394928, -0.02839279, -0.00915527,
        -0.05337811, -0.00292492,  0.01215386, -0.03006744, -0.01864243,
         0.05606079,  0.01688766, -0.01224995, -0.02986145, -0.00167847,
         0.0299015 , -0.03596306, -0.0301609 ,  0.06640148, -0.03706074,
        -0.03068161,  0.07945681, -0.04376984, -0.07824707,  0.02370071,
        -0.04793549, -0.04295349,  0.10340214, -0.0

In [69]:
featureVec = np.zeros((1, 300),dtype="float32")

for doc in text[:2]:
    nwords = 0
    for word in doc:
        if word in set(model.index2word):
            nwords += 1
            featureVec = np.add(featureVec,model[word])
    featureVec = np.divide(featureVec, nwords)
            
            

In [32]:
featureVec

array([[-0.18475981,  0.13650087, -0.01074219, ..., -0.03607887,
        -0.09047113,  0.16226836],
       [-0.18475981,  0.13650087, -0.01074219, ..., -0.03607887,
        -0.09047113,  0.16226836],
       [-0.18475981,  0.13650087, -0.01074219, ..., -0.03607887,
        -0.09047113,  0.16226836],
       ..., 
       [-0.18475981,  0.13650087, -0.01074219, ..., -0.03607887,
        -0.09047113,  0.16226836],
       [-0.18475981,  0.13650087, -0.01074219, ..., -0.03607887,
        -0.09047113,  0.16226836],
       [-0.18475981,  0.13650087, -0.01074219, ..., -0.03607887,
        -0.09047113,  0.16226836]], dtype=float32)

In [34]:
featureVec = np.zeros((1, 300),dtype="float32")

featureVec = [[np.add(featureVec, model[word]) for doc in text if word in set(model.index2word)] for doc in text[:3]]

featureVec = [np.divide(featureVec, len(doc)) for doc in text]
print("DONE")

KeyboardInterrupt: 